In [1]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
# Surprise Package
import surprise

In [2]:
# Read data
train_df = pd.read_pickle("../../train_df.pkl")
test_df = pd.read_pickle("../../test_df.pkl")

In [4]:
# Create Train/Test flag
train_df["Set"] = "Train"
test_df["Set"] = "Test"

### Merge datasets

In [5]:
data = pd.concat([train_df, test_df], sort = False)

In [7]:
# Explore dataset
print(data.shape)
data.head(5)

(290639, 21)


,reviewID,overall,verified,reviewTime,reviewerID,productID,reviewText,summary,vote,style,...,title,brand,rank,main_cat,description,also_buy,also_view,feature,numberOfReviews,Set
473590,473590,5,False,2016-07-16,A2KJVIU82ZUSBA,B0186D1UQ0,Fantastic,Best Remake I've Ever Played,0,{'Format:': ' Video Game'},...,Odin Sphere Leifthrasir - PlayStation 4 Standa...,by Atlus,">#4,665 in Video Games (See Top 100 in Video G...",Video Games,"[""ATLUS x Vanillaware, the team behind the hit...","['B077Y7313L', 'B0794Y8FLW', 'B01HQGWBZ2', 'B0...","['B077Y7313L', 'B0794Y8FLW', 'B017GY06UQ', 'B0...","['HD Art Upgrade: Known for its beautiful, 2D ...",26,Train
128446,128446,5,True,2014-07-18,A3GENOZU8QJZAZ,B000R39GPA,Great game,PS3,0,"{'Edition:': ' Standard', 'Platform:': ' PlayS...",...,Star Wars: The Force Unleashed,by LucasArts,">#9,056 in Video Games (See Top 100 in Video G...",Video Games,['The Star Wars saga will continue in Star War...,"['B0030F1DOO', 'B003FMUMBM', 'B00EY3F8LU', 'B0...","['B01BCMBP44', 'B0030F1DOO', 'B00EY3F8LU', 'B0...",['Confront and associate with familiar faces f...,140,Train
356264,356264,3,True,2015-07-03,ADE6VBLJEOKFC,B00DHF39HQ,Its a game with stuff thats really boring to s...,Buy a car.,2,{'Format:': ' Video Game'},...,Elder Scrolls Online: Tamriel Unlimited - Play...,by Bethesda,">#5,896 in Video Games (See Top 100 in Video G...",Video Games,"[""The award winning Elder Scrolls saga goes on...","['B01MSDDCUI', 'B07BJ66LRB', 'B01I41HEGY', 'B0...","['B07BJ66LRB', 'B01I41HEGY', 'B01MSDDCUI', 'B0...","['Unlimited Adventures, No Subscription Requir...",135,Train
396581,396581,5,True,2014-12-25,AGS25M1ED8EGZ,B00K848IH0,love it,great,0,"{'Edition:': ' Alpha Sapphire', 'Platform:': '...",...,Pokemon Alpha Sapphire - Nintendo 3DS,by Nintendo,>#837 in Video Games (See Top 100 in Video Gam...,Video Games,"[""Soar high above the Hoenn region on an unfor...","['B0053B66KE', 'B0053B5RGI', 'B071WNTGFY', 'B0...","['B0053B5RGI', 'B00R11EPAC', 'B071JQQ9PK', 'B0...",['Games in 2D. Some areas also playable in 3D....,267,Train
244246,244246,4,True,2011-07-15,A1CZT2GDDO4GC1,B003O6G1L4,The main people who are really DOWNING this ga...,"It's not that bad of a game, but does lack a l...",5,"{'Edition:': ' Standard', 'Platform:': ' PlayS...",...,Dungeon Siege III - Playstation 3,by Square Enix,">#19,535 in Video Games (See Top 100 in Video ...",Video Games,"[""Torn asunder, the delicate balance of power ...","['B00IQCRO3K', 'B001BTPRKC', 'B000HCJD68', 'B0...","['B00IQCRO3K', 'B007V9QKH0', 'B00B59MXGY', 'B0...","['For the first time, play DUNGEON SIEGE on hi...",43,Train


### Drop lines without recommendation

In [59]:
# Missing recommendations
sum(data["also_buy"].isna())

13311

In [60]:
# Percentage
sum(data["also_buy"].isna())/len(data)*100

4.579908408713215

In [61]:
# Drop rows with no recommendation
data.dropna(axis = 0, subset = ["also_buy"], inplace = True)

In [62]:
# Check dropna
sum(data["also_buy"].isna())

0

In [63]:
# New length of the data
len(data)

277328

### Obtaining unique list of videogames

In [80]:
# Unique products
games_list = set(data["productID"].unique())

In [81]:
# Explore list of unique videogames
len(games_list)

9322

### Subsetting recommendations only to videogames

In [72]:
# Convert string to list
data["amz_recommendation"] = data["also_buy"].apply(lambda x: ast.literal_eval(x))

In [84]:
# Keep only products that are videogames
data["game_recommendation"] = data["amz_recommendation"].apply(lambda x: set(x).intersection(games_list))

In [87]:
# Count recommended games per row
data["n_recommended_games"] = data["game_recommendation"].apply(lambda x: len(x))

In [88]:
# Count recommended products per row
data["n_recommended_products"] = data["amz_recommendation"].apply(lambda x: len(x))

In [96]:
# Check that they are mostly different
sum(data["n_recommended_games"] != data["n_recommended_products"])

273635

### Checking for sufficient recommendations

In [98]:
# Check for how many rows we have at least 10 recommendations
sum(data["n_recommended_games"] >= 10)

235594

In [101]:
# As %
sum(data["n_recommended_games"] >= 10)/len(data)*100

84.95139329602492

In [99]:
# Check for how many rows we have at least 15 recommendations
sum(data["n_recommended_games"] >= 15)

223381

In [102]:
# As %
sum(data["n_recommended_games"] >= 15)/len(data)*100

80.54758264582011